<h1 align="center"><strong>gbapy</strong></h1>
<h5 align="center">Growth Balance Analysis for Python</h5>
<h3 align="center"><strong>Tutorial:</strong> standard organization of a GBA model in CSV format</h3>

## Table of contents
* [1) Files organisation](#files_organisation)
* [2) Detailed content](#detailed_content)
  * [2.1) File `Infos.csv`](#infos)
  * [2.2) File `directions.csv`](#infos)
  * [2.3) File `M.csv`](#M)
  * [2.4) File `KM_forward.csv`](#km_forward)
  * [2.5) File `kcat.csv`](#kcat)
  * [2.6) File `conditions.csv`](#conditions)

## 1) Files organisation <a class="anchor" id="files_organisation"></a>

The default format developed for GBA models is a set of CSV files, containing each a variable of the model. The user may refer to <a href="https://doi.org/10.1371/journal.pcbi.1011156" target="_blank">Dourado et al. (2023)</a> to get the detail of the construction of the GBA model.


The list of the minimal set of CSV files (describing a model with irreversible Michaelis-Menten forward reactions only) is:
- `Infos.csv`: A description of the model, including its name,
- `directions.csv`: A list of reactions' directions (`forward`, `backward`, or `reversible`),
- `M.csv`: The mass fraction matrix $\mathbf{M}$ (in g.L<sup>-1</sup>),
- `KM_forward.csv`: The matrix of Michaelis constants $K_\text{M}$ for forward reactions (in g.L<sup>-1</sup>),
- `kcat.csv`: Vectors of turnover rates $k_\text{cat}$ for forward and backward directions (in h<sup>-1</sup>),
- `conditions.csv`: The list of external conditions and total density $\rho$.

Additional files can be included depending on modeling choices:
- `KM_backward.csv`: The matrix of Michaelis constants $K_\text{M}$ for backward reactions (in g.L<sup>-1</sup>),
- `KA.csv`: The matrix of activation constants $K_\text{A}$ for forward reactions <strong>only</strong> (in g.L<sup>-1</sup>),
- `KI.csv`: The matrix of inhibition constants $K_\text{I}$ for forward reactions <strong>only</strong> (in g.L<sup>-1</sup>).

Note that activation and inhibition can only be used for irreversible forward reactions.

All the CSV files of a model must be included in one folder (usually, with the name of the model). <em>E.g.</em> for a model named "A":

      └── A
          ├── Infos.csv
          ├── directions.csv
          ├── M.csv
          ├── KM_forward.csv
          ├── kcat.csv
          └── conditions.csv

When calling the function `read_csv_model("A")`, <strong>gbapy</strong> will parse the CSV files and automatically decide of the kinetic model (irreversible/reversible Michaelis-Menten kinetics, with or without activation or/and inhibition).

## 2) Detailed content <a class="anchor" id="detailed_content"></a>
For the following, we will use the toy model "A", furnished with <strong>gbapy</strong>, to review the content of the main CSV files.
To locate the toy model, we can use the function `gbapy.get_toy_model_path("A")`:

In [1]:
# Import gbapy, os and pandas modules
import gbapy
import os
import pandas as pd

In [2]:
# Get toy model A path
A_path = gbapy.get_toy_model_path("A")
A_path

'/usr/local/lib/python3.12/site-packages/gbapy/data/toy_models/A'

The content of toy model A folder is the following:

In [3]:
os.system("ls -l "+A_path)

total 64
-rw-r--r--  1 charlesrocabert  admin   353 Oct 19 15:47 Infos.csv
-rw-r--r--  1 charlesrocabert  admin    44 Oct 19 15:47 KM_forward.csv
-rw-r--r--  1 charlesrocabert  admin    44 Oct 19 15:47 M.csv
-rw-r--r--  1 charlesrocabert  admin   466 Oct 19 15:47 conditions.csv
-rw-r--r--  1 charlesrocabert  admin    49 Oct 19 15:47 directions.csv
-rw-r--r--  1 charlesrocabert  admin    35 Oct 19 15:47 f0.csv
-rw-r--r--  1 charlesrocabert  admin    54 Oct 19 15:47 kcat.csv
-rw-r--r--  1 charlesrocabert  admin  1805 Oct 19 15:47 optimums.csv


0

We note that two additional files are present:
- `optimums.csv`: contains the list of optima for each condition of the model,
- `f0.csv`: initial sub-optimal solution for the $\mathbf{f}$ vector.

This files are generated when using <strong>gbapy</strong> routine functions (see tutorial `basic_manipulations_tutorial.ipynb`).

### 2.1) File `Infos.csv` <a class="anchor" id="infos"></a>

The file `Infos.csv` contains various information about the model (units, file description, ...).

$\Rightarrow$ Note that the variable "Name" is <strong>mandatory</strong>, as <strong>gbapy</strong> relies on it to name the loaded model.

In [4]:
pd.read_csv(A_path+"/Infos.csv", sep=";", header=None)

,0,1
0,Name,A
1,Description,Simplest GBA model with two reactions
2,Units,Km: g/L
3,Units,kcat: 1/h ([mass of products]/[mass of protein...
4,Units,rho: g/L
5,Files,M: Mass fraction matrix
6,Files,KM_forward: Forward Michaelis constant matrix
7,Files,kcat: Turnover numbers
8,Files,conditions: Value of rho and external concentr...


### 2.2) File `directions.csv` <a class="anchor" id="directions"></a>

This file contains the list of directions for all reactions.

In [5]:
pd.read_csv(A_path+"/directions.csv", sep=";")

,reaction,direction
0,rxn1,forward
1,Ribosome,forward


### 2.3) File `M.csv` <a class="anchor" id="M"></a>

This file contains the mass fraction matrix (see <a href="https://doi.org/10.1371/journal.pcbi.1011156" target="_blank">Dourado et al., 2023</a>). Metabolites are in row, reactions in columns. The last row corresponds to total protein amount. The last column corresponds to the ribosome reaction, producing the total protein amount.

All metabolites starting with `x_` are external metabolites with constant concentration.

In [6]:
pd.read_csv(A_path+"/M.csv", sep=";")

,Unnamed: 0,rxn1,Ribosome
0,x_G,-1,0
1,AA,1,-1
2,Protein,0,1


### 2.4) File `KM_forward.csv` <a class="anchor" id="km_forward"></a>

This file contains the matrix of forward Michaelis constants $K_\text{M}$. This file is mandatory for the minimal kinetics (<em>i.e.</em> irreversible forward Michaelis-Menten kinetics).
One can add a second matrix for backward Michaelis constants (`KM_backward.csv`) if backward or reversible reactions are present in the model.

In [7]:
pd.read_csv(A_path+"/KM_forward.csv", sep=";")

,Unnamed: 0,rxn1,Ribosome
0,x_G,1,0.0
1,AA,0,8.3
2,Protein,0,0.0


### 2.5) File `kcat.csv` <a class="anchor" id="kcat"></a>

This file contains the vectors of forward and backward turnover rates $k_\text{cat}$. If all reactions are forward irreversible, all backward values will be zero (which is the case here).

In [8]:
pd.read_csv(A_path+"/kcat.csv", sep=";")

,Unnamed: 0,rxn1,Ribosome
0,kcat_forward,10,4.55
1,kcat_backward,0,0.00


### 2.6) File `conditions.csv` <a class="anchor" id="conditions"></a>

This file contains the list of external conditions. Each condition contains:
- The concentration of every external metabolite,
- The total density $\rho$ (here 340 g.L<sup>-1</sup> for <em>Escherichia coli</em>).

In [9]:
pd.read_csv(A_path+"/conditions.csv", sep=";")

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,rho,340,340.000000,340.000000,340.00000,340.000000,340.000000,340.00000,340.000000,340.000000,...,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.00000,340.00000
1,x_G,100,66.666667,44.444444,29.62963,19.753086,13.168724,8.77915,5.852766,3.901844,...,0.228366,0.152244,0.101496,0.067664,0.045109,0.030073,0.020049,0.013366,0.00891,0.00594
